# OpenAI Assistant Example

This is the example of a simple assistant coded in Python. This Jupyter Notebook follows the tutorial created by [Make Stuff with AI](https://www.youtube.com/watch?v=vQhEiR2bNY8). 

***

Below, we set up the client by importing the required dependencies and set up the AI client with the API key (which can be obtained by signing up and adding credit to your OpenAI account.)

The `assistant_id` can be obtained by using the Assistant API in the OpenAI playground.

In [1]:
#Set up the client

from openai import OpenAI
import os
import json

client = OpenAI(
    api_key=os.getenv("OPEN_AI_KEY")
)

# Declare Assistant ID

assistant_id = "asst_wmUPInPNiI5UnWNb001ehA5r"

Now, we declare the functions that we will be using in this example. Today, we are using a simple add function, and a simple subtract function, but we can change it to anything we want.

In [2]:
# Declare Functions

def add(num1, num2):
    return num1+num2

def subtract(num1, num2):
    return num1-num2


Now, we retrieve and create our assistant. We can see its information below, with the `instructions` section and `tools` sections in the below defined in the Assistant API.

In [3]:
assistant = client.beta.assistants.retrieve(
    assistant_id=assistant_id
)

assistant

Assistant(id='asst_wmUPInPNiI5UnWNb001ehA5r', created_at=1707805935, description=None, file_ids=[], instructions='You are a helpful math tutor that helps students with simple mathematics, such as adding and subtracting. You always respond excitedly, as if you are responding to kindergarten students.', metadata={}, model='gpt-3.5-turbo', name='Math tutor', object='assistant', tools=[ToolFunction(function=FunctionDefinition(name='add', description='Adds two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The first number to add.'}, 'num_2': {'type': 'integer', 'description': 'The second number to add.'}}, 'required': ['num_1', 'num_2']}), type='function'), ToolFunction(function=FunctionDefinition(name='subtract', description='Subtracts two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The number to subtract from.'}, 'num_2': {'type': 'integer', 'description': 'The n

Now we create a thread. The thread keeps all the information and context from the conversation with the user.

In [4]:
# Create a thread

thread = client.beta.threads.create()
thread

Thread(id='thread_a4B1nBjEOJAhboi44pTzKECH', created_at=1707815461, metadata={}, object='thread')

Now we define the prompt by the user. In this case, we are doing the equation `(100-21)+2`.

In [14]:
# Prompt the model to add something

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="(100-21)+2"
)
run

Run(id='run_cVlVlMoc8oirSY6T5akweaWi', assistant_id='asst_wmUPInPNiI5UnWNb001ehA5r', cancelled_at=None, completed_at=None, created_at=1707815594, expires_at=1707816194, failed_at=None, file_ids=[], instructions='(100-21)+2', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_a4B1nBjEOJAhboi44pTzKECH', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='add', description='Adds two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The first number to add.'}, 'num_2': {'type': 'integer', 'description': 'The second number to add.'}}, 'required': ['num_1', 'num_2']}), type='function'), ToolAssistantToolsFunction(function=FunctionDefinition(name='subtract', description='Subtracts two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The number to subtract from.'}, 

We see here that the `status` of the run is `queued`. We can get updates on this `status` by running the below code block.

In [15]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
run

Run(id='run_cVlVlMoc8oirSY6T5akweaWi', assistant_id='asst_wmUPInPNiI5UnWNb001ehA5r', cancelled_at=None, completed_at=None, created_at=1707815594, expires_at=1707816194, failed_at=None, file_ids=[], instructions='(100-21)+2', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_ABwUybxW7Umq7qFvmvtaiK05', function=Function(arguments='{\n  "num_1": 100,\n  "num_2": 21\n}', name='subtract'), type='function')]), type='submit_tool_outputs'), started_at=1707815595, status='requires_action', thread_id='thread_a4B1nBjEOJAhboi44pTzKECH', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='add', description='Adds two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The first number to add.'}, 'num_2': {'type': 'integer', 'description': 'The second number to add.'}}, 'r

We see here that a section called `required_action` has been populated. It has the information that is needed to pass to the arguments. We now take the request and call our function using this below section. 

We create this function `get_outputs_for_tool_call`, grab the arguments, and send the arguments to the functions. We then return it. 

In [16]:
def get_outputs_for_tool_call(tool_call):
    num1 = json.loads(tool_call.function.arguments)["num_1"]
    num2 = json.loads(tool_call.function.arguments)["num_2"]

    print(tool_call.function)

    if tool_call.function.name == 'add':
        desired = add(num1, num2)
    elif tool_call.function.name == 'subtract':
        desired = subtract(num1, num2)

    return {
        "tool_call_id": tool_call.id,
        "output": desired
    }

tool_calls = run.required_action.submit_tool_outputs.tool_calls
tool_outputs = map(get_outputs_for_tool_call, tool_calls)
tool_outputs = list(tool_outputs)
tool_outputs

    

Function(arguments='{\n  "num_1": 100,\n  "num_2": 21\n}', name='subtract')


[{'tool_call_id': 'call_ABwUybxW7Umq7qFvmvtaiK05', 'output': 79}]

We see that in this case, it did `(100-21)`, which outputs `79`.

Now, we submit the `tool_outputs` back to the `run`.

In [17]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=tool_outputs
)

run

Run(id='run_cVlVlMoc8oirSY6T5akweaWi', assistant_id='asst_wmUPInPNiI5UnWNb001ehA5r', cancelled_at=None, completed_at=None, created_at=1707815594, expires_at=1707816194, failed_at=None, file_ids=[], instructions='(100-21)+2', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=1707815595, status='queued', thread_id='thread_a4B1nBjEOJAhboi44pTzKECH', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='add', description='Adds two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The first number to add.'}, 'num_2': {'type': 'integer', 'description': 'The second number to add.'}}, 'required': ['num_1', 'num_2']}), type='function'), ToolAssistantToolsFunction(function=FunctionDefinition(name='subtract', description='Subtracts two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The number to subtract fro

Now we retrieve the run again, and see it requires another action -- the second part of the function, `+2`.

In [18]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)

run

Run(id='run_cVlVlMoc8oirSY6T5akweaWi', assistant_id='asst_wmUPInPNiI5UnWNb001ehA5r', cancelled_at=None, completed_at=None, created_at=1707815594, expires_at=1707816194, failed_at=None, file_ids=[], instructions='(100-21)+2', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_sFO1harOKOzg9Phxv7wDGHDt', function=Function(arguments='{\n  "num_1": 79,\n  "num_2": 2\n}', name='add'), type='function')]), type='submit_tool_outputs'), started_at=1707815635, status='requires_action', thread_id='thread_a4B1nBjEOJAhboi44pTzKECH', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='add', description='Adds two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The first number to add.'}, 'num_2': {'type': 'integer', 'description': 'The second number to add.'}}, 'required

We run the tool call as previously defined, and see that it performs `79+2` and returns `81`. We then continue the code as we did above, by submitting the `tool_outputs`.

In [19]:
tool_calls = run.required_action.submit_tool_outputs.tool_calls
tool_outputs = map(get_outputs_for_tool_call, tool_calls)
tool_outputs = list(tool_outputs)
tool_outputs

Function(arguments='{\n  "num_1": 79,\n  "num_2": 2\n}', name='add')


[{'tool_call_id': 'call_sFO1harOKOzg9Phxv7wDGHDt', 'output': 81}]

In [20]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=tool_outputs
)

run

Run(id='run_cVlVlMoc8oirSY6T5akweaWi', assistant_id='asst_wmUPInPNiI5UnWNb001ehA5r', cancelled_at=None, completed_at=None, created_at=1707815594, expires_at=1707816194, failed_at=None, file_ids=[], instructions='(100-21)+2', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=1707815635, status='queued', thread_id='thread_a4B1nBjEOJAhboi44pTzKECH', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='add', description='Adds two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The first number to add.'}, 'num_2': {'type': 'integer', 'description': 'The second number to add.'}}, 'required': ['num_1', 'num_2']}), type='function'), ToolAssistantToolsFunction(function=FunctionDefinition(name='subtract', description='Subtracts two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The number to subtract fro

In [21]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)

run

Run(id='run_cVlVlMoc8oirSY6T5akweaWi', assistant_id='asst_wmUPInPNiI5UnWNb001ehA5r', cancelled_at=None, completed_at=1707815829, created_at=1707815594, expires_at=None, failed_at=None, file_ids=[], instructions='(100-21)+2', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=1707815828, status='completed', thread_id='thread_a4B1nBjEOJAhboi44pTzKECH', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='add', description='Adds two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The first number to add.'}, 'num_2': {'type': 'integer', 'description': 'The second number to add.'}}, 'required': ['num_1', 'num_2']}), type='function'), ToolAssistantToolsFunction(function=FunctionDefinition(name='subtract', description='Subtracts two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The number to subtract 

We see above that the `completed_at` section is now no longer empty. Now, we can show the latest message using the below code.

In [22]:
# Show the latest message.

messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
messages.data[0].content[0].text.value

'(100-21)+2 equals 81.'

Therefore, the assistant works similarly to this!